## Setup

In [2]:
# importing libraries
import pandas as pd # data manipulation

from tika import parser # pdf bill text extraction
from wwo_hist import retrieve_hist_data # weather data api
import re # text formatting with regex

import arrow # date formatting
import datetime as dt # date formatting

import env # encoded data

# show all columns
pd.set_option('display.max_columns', None)

In [3]:
# storing august and september bill filepaths (september is for testing)
filePath_aug = 'pdfs/2022-08-25_bill.pdf'
filePath_sep = 'pdfs/2022-09-27_bill.pdf'

## Text Extraction Function

In [4]:
# creating function for extracting text 
def extract_pdf(filePath):
    '''
    Argument(s)
        - filePath (str): path to the pdf file to be extracted
    Returns 
        - text (str): extracted pdf text
    '''
    
    text = parser.from_file(filePath)['content']
    print(f'Text extraction from {filePath} complete.')
    
    return text

## PDFs Text Extraction Function

In [5]:
def looped_FileExtraction(filePath_list):
    '''
    This function takes in a list of pdf file paths, loops through that list and stores the extracted
    text as dictionary values, creating associated key names based on the pdf name (bill month), and 
    reading that dictionary to pandas df.
    
    Args:
        - filePath_list (list): list of pdf file paths as strings
    
    Returns:
        - df: 
    '''
    # initiating empty dictionary to store extracted text as values and month labels as keys
    text_dict = {}

    # initializing empty list to store dictionary keys
    text_list = []

    # looping through each file path to extract text using extract_pdf() function
    for file in filePath_list:
        text_dict[f'{arrow.get(file[7:15], "YY-MM-DD".format("YYY-MM-DD"))}'[:10]] = extract_pdf(file)
        text_list.append(f'text_{arrow.get(file[7:15], "YY-MM-DD".format("YYY-MM-DD"))}')
        
    # writing dictionary to df
    df = pd.DataFrame.from_dict(text_dict, orient='index', columns = ['text'])
    df.reset_index(inplace = True)
    df.rename(columns = {'index':'bill_date'}, inplace = True)
    
    return df

## Text Parsing Function

In [11]:
def text_parse(df):
    '''
    
    '''
    # initalizing empty lists to store values from loop for df
    bill_dates = []
    bills_starts = []
    bills_ends = []
    bill_periods = []
    
    meters = []
    consumptions = []
    peaks = []
    peak_alls = []
    
    consumption_rates = []
    peak_rates = []
    fuel_rates = []
    regulatory_rates = []
    
    service_charges = []
    consumption_charges = []
    peak_charges = []
    fuel_charges = []
    regulatory_charges = []
    total_bills = []
    
    # looping through each bill
    for text in df.text:
    
        #################
        # BILLING INFO
        #################
        bill_date = extract_text(text, f'{env.zip}\n\n', '\n\nOn or Before')
        bill_dates.append(bill_date)
        print(f'Bill Date: {bill_date}')
        
        billing_data = extract_text(text, 'Billing Period ', '\nYour next')
        
        bill_start = arrow.get(billing_data.split('-')[0].strip(), 'MMM DD, YYYY').format('YYYY-MM-DD')
        bills_starts.append(bill_start)
        print(f'Bill Start: {bill_start}')
        
        bill_end = arrow.get(billing_data.split('-')[1].strip(), 'MMM DD, YYYY').format('YYYY-MM-DD')
        bills_ends.append(bill_end)
        print(f'Bill End: {bill_end}')
        
        bill_period = (pd.to_datetime(bill_end) - pd.to_datetime(bill_start)).days + 1
        bill_periods.append(bill_period)
        print(f'Bill Period: {bill_period} days')

        #################
        # BASE AMOUNTS
        #################
        list_ = extract_text(text, 'R-', '#6271330').replace(')', '').replace(',', '').split()
        
        meter = int(list_[1][:5])
        meters.append(meter)
        print(f'Meter Reading: {meter}')
        
        consumption = int(list_[2])
        consumptions.append(consumption)
        print(f'Consumption: {consumption}')
        
        peak = int(extract_text(text, 'Peak Capacity Charge ', '\n\n').split('\n')[0].split()[0])
        peaks.append(peak)
        print(f'Actual Peak Consumption: {peak}')
        
        peak_all = int(consumption - 600)
        peak_alls.append(peak_all)
        print(f'Peak Consumption (all): {peak_all}')

        #################
        # RATES
        #################
        
        consumption_rates.append()
        
        peak_rates.append()
        
        fuel_rates.append()
        
        regulatory_rates.append()

        #################
        # CHARGES
        #################
        
        service_charges.append()
        
        consumption_charges.append()
        
        peak_charges.append()
        
        fuel_charges.append()
        
        regulatory_charges.append()
        
        total_bills.append()
        